In [ ]:
#| hide

%load_ext autoreload
%autoreload 2

# Google's MCP Toolbox

> In this notebook, we will explore how to configure and use various tools available in Google's MCP Toolbox. These are defined in `tools.yaml` configuration file.

In [ ]:
#| default_exp toolbox

In [ ]:
#| export

from toolbox_core import ToolboxSyncClient
from agents import function_tool
from thucy.configuration import config

In [ ]:
#| export

class GenAIToolboxMCP:
    """Factory for managing a shared ToolboxSyncClient and loading toolsets."""
    def __init__(self):
        self.toolset = None

    def set_specific_toolset(self, toolset: str):
        self.toolset = toolset

    def _load_raw_toolset(self, toolset_name: str):
        """Load a toolset by name and wrap it with openai sdk tool tool func."""

        # If a specific database is set, modify the toolset name accordingly.
        # This allows for specific database toolsets to be loaded, instead
        # of the "big" generic ones.
        if self.toolset:
            toolset_name = f"{self.toolset}-{toolset_name}"

        toolset = self.client.load_toolset(toolset_name)

        return toolset

    def load_toolset(self, toolset_name: str):
        """Load a toolset by name and wrap it with openai sdk tool tool func."""
        return [function_tool(tool) for tool in self._load_raw_toolset(toolset_name)]
    
    def connect(self):
        self.client = ToolboxSyncClient(config.genai_server_url)
    
    def close(self):
        self.client.close()

We first create a singleton instance of `GenAIToolboxMCP`.

In [ ]:
#| exports
#| notest
genai_mcp = GenAIToolboxMCP()

Then, we preconfigure the tools to redirect to the **toolsets** with the prefix of **seattle**. For example, after the following command, the SQL expert agent will subscribe to the tooset **seattle-sql**.

In [ ]:
#| notest
genai_mcp.set_specific_toolset("seattle")

In [ ]:
#| notest
genai_mcp.connect()

Let's load some tools and see what is going on. First, we will load **sql** tools:

In [ ]:
#| notest
sql_tools = genai_mcp._load_raw_toolset("sql")

The length of the loaded tools should be 1, as we have 1 SQL toolsets in the `seattle` toolset: `postgres_seattle_execute_sql`.

In [ ]:
#| notest
len(sql_tools)

1

Let's run some SQL using a tool. **They are directly callable**!

In [ ]:
#| notest
sql_tools[0]("select * from crime_data limit 1;")

'[{"beat":"W2","block_address":"59XX BLOCK OF 45TH AVE SW","latitude":"47.549439","longitude":"-122.389912","neighborhood":"MORGAN","nibrs_crime_against_category":"PROPERTY","nibrs_group_ab":"A","nibrs_offense_code":"23F","nibrs_offense_code_description":"Theft From Motor Vehicle","offense_category":"PROPERTY CRIME","offense_date":"2024-07-05T22:00:00Z","offense_id":57458328658,"offense_sub_category":"LARCENY-THEFT","precinct":"Southwest","report_datetime":"2024-07-07T07:00:10Z","report_number":"2024-934604","reporting_area":"1858","sector":"W","shooting_type_group":"-"}]'

Let's load the **schema** toolset and play around with it.

In [ ]:
#| notest
schema_tools = genai_mcp._load_raw_toolset("schema")

It is going to show 1 tools again. 

In [ ]:
#| notest
len(schema_tools)

1

In [ ]:
#| notest
schema_tools[0]('crime_data', 'simple')

'[{"object_details":{"name":"crime_data"},"object_name":"crime_data","schema_name":"public"}]'

In [ ]:
#| notest
schema_tools[0]('crime_data', 'detailed')

'[{"object_details":{"columns":[{"column_comment":null,"column_default":null,"column_name":"report_number","data_type":"text","is_not_nullable":false,"ordinal_position":1},{"column_comment":null,"column_default":null,"column_name":"report_datetime","data_type":"timestamp without time zone","is_not_nullable":false,"ordinal_position":2},{"column_comment":null,"column_default":null,"column_name":"offense_id","data_type":"bigint","is_not_nullable":false,"ordinal_position":3},{"column_comment":null,"column_default":null,"column_name":"offense_date","data_type":"timestamp without time zone","is_not_nullable":false,"ordinal_position":4},{"column_comment":null,"column_default":null,"column_name":"nibrs_group_ab","data_type":"text","is_not_nullable":false,"ordinal_position":5},{"column_comment":null,"column_default":null,"column_name":"nibrs_crime_against_category","data_type":"text","is_not_nullable":false,"ordinal_position":6},{"column_comment":null,"column_default":null,"column_name":"offens

We can also load the toolset used by OpenAI Agents (wrapped in tool funcs)

In [ ]:
#| notest
genai_mcp.load_toolset('sql')

[FunctionTool(name='postgres_seattle_execute_sql', description='Executes SQL queries on the PostgreSQL Seattle database. The queries must be PostgreSQL-compatible.', params_json_schema={'properties': {'sql': {'description': 'The sql to execute.', 'title': 'Sql', 'type': 'string'}}, 'required': ['sql'], 'title': 'postgres_seattle_execute_sql_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)]

In [ ]:
#| notest
genai_mcp._load_raw_toolset('sql')

[<toolbox_core.sync_tool.ToolboxSyncTool>]

In [ ]:
#| notest
genai_mcp.close()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()